# Создание дашборда по пользовательским событиям

## Описание

Представим, что мы работаем аналитиком в Яндекс.Дзене. Почти всё наше время занимает анализ пользовательского взаимодействия с карточками статей.
Каждую карточку определяют её тема и источник (у него тоже есть тема). Примеры тем: «Красота и здоровье», «Россия», «Путешествия».
Пользователей системы характеризует возрастная категория. Скажем, «26-30» или «45+».

Есть три способа взаимодействия пользователей с системой:
* Карточка отображена для пользователя (show);
* Пользователь кликнул на карточку (click);
* Пользователь просмотрел статью карточки (view).

Каждую неделю начинающие менеджеры задают нам одни и те же вопросы: 
* Cколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
* Как много событий генерируют источники с разными темами?
* Как соотносятся темы карточек и темы источников?

На шестую неделю работы мы решаем, что процесс пора автоматизировать: нужно сделать дашборд.
Пообщавшись с менеджерами и администраторами баз данных, мы написали краткое ТЗ:
* Бизнес-задача: анализ взаимодействия пользователей с карточками Яндекс.Дзен;
* Насколько часто предполагается пользоваться дашбордом: не реже, чем раз в неделю;
* Кто будет основным пользователем дашборда: менеджеры по анализу контента;
* Состав данных для дашборда:
   - История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
   - Разбивка событий по темам источников;
   - Таблица соответствия тем источников темам карточек;
* По каким параметрам данные должны группироваться:
   - Дата и время;
   - Тема карточки;
   - Тема источника;
   - Возрастная группа;
* Характер данных:
   - История событий по темам карточек — абсолютные величины с разбивкой по минутам;
   - Разбивка событий по темам источников — относительные величины (% событий);
   - Соответствия тем источников темам карточек - абсолютные величины;
* Важность: все графики имеют равную важность;
* Частота обновления данных: один раз в сутки, в полночь по UTC;
* Какие графики должны отображаться и в каком порядке, какие элементы управления должны быть на дашборде (макет дашборда).

Источники данных для дашборда:
* record_id — первичный ключ,
* item_topic — тема карточки,
* source_topic — тема источника,
* age_segment — возрастной сегмент,
* dt — дата и время,
* visits — количество событий.

## Выгрузка данных

Выгрузим данные из таблицы и сохраним ее в файл 'dash_visits.csv'.

In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# импортируем библиотеки
import sys
import getopt
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string) 

In [3]:
# получаем сырые данные
query = '''
            SELECT * FROM dash_visits
        '''

dash_visits = pd.io.sql.read_sql(query, con = engine) 
dash_visits['dt'] = pd.to_datetime(dash_visits['dt'])

In [4]:
dash_visits.to_csv(r'C:\\Users\\foksi\Downloads\dash_visits.csv')